In [4]:
import cv2
import numpy as np
import os
import random
from glob import glob

# Define paths for the input folder and output results folder
input_folder = 'C:/Users/Haseeb Raza/Desktop/MIC_3/set4'
output_folder = 'C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4'
os.makedirs(output_folder, exist_ok=True)

# Initialize SIFT detector with max 800 keypoints
noOfmaxKeypts = 800
sift = cv2.xfeatures2d.SIFT_create(noOfmaxKeypts)

# Process each image in the set4 folder
image_paths = glob(os.path.join(input_folder, '*.png'))
for image_path in image_paths:
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load {image_path}")
        continue

    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if len(keypoints) < 200:
        print(f"Warning: {image_path} has less than 200 keypoints.")
        
    # Select 200 random keypoints for visualization
    random_keypoints = random.sample(keypoints, min(200, len(keypoints)))

    # Draw the 200 random keypoints on the image
    image_with_keypoints = cv2.drawKeypoints(
        image, random_keypoints, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS
    )

    # Generate a unique output filename based on the input image name
    image_filename = os.path.basename(image_path).replace('.png', '_featurePoints.jpg')
    output_path = os.path.join(output_folder, image_filename)
    
    # Save the resulting image with the keypoints drawn
    cv2.imwrite(output_path, image_with_keypoints)
    print(f"Processed and saved: {output_path}")


Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_10_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_11_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_12_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_1_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_2_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_3_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_4_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_5_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_6_100_AML_featurePoints.jpg
Processed and saved: C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_7_100

In [2]:
import cv2
import numpy as np
import os

# Define paths for images
image_folder = 'C:/Users/Haseeb Raza/Desktop/MIC_3/set4/'
output_folder = 'C:/Users/Haseeb Raza/Desktop/MIC_3/Results/'
os.makedirs(output_folder, exist_ok=True)

# Load images from the folder
images = [cv2.imread(image_folder + f"set4_{i}_100_AML.png") for i in range(1, 13)]

# Initialize SIFT detector
noOfmaxKeypts = 800
sift = cv2.xfeatures2d.SIFT_create(noOfmaxKeypts)

def extract_features_and_descriptors(img):
    # Step 1: Detect keypoints and descriptors
    keypoints, descriptors = sift.detectAndCompute(img, None)
    
    # Create an array of [x, y, radius, angle] for each keypoint
    features = np.array([[kp.pt[0], kp.pt[1], kp.size, kp.angle] for kp in keypoints]).T
    
    return keypoints, descriptors, features

# Extract SIFT features for each image in the set
all_keypoints = []
all_descriptors = []
for img in images:
    keypoints, descriptors, features = extract_features_and_descriptors(img)
    all_keypoints.append(keypoints)
    all_descriptors.append(descriptors)
    
    # Visualize and save the SIFT feature points for one image
    img_with_keypoints = cv2.drawKeypoints(img, keypoints[:200], None, color=(0, 255, 0), flags=0)
    cv2.imwrite(f"{output_folder}/set4_featurePoints.jpg", img_with_keypoints)

# Match features using Euclidean Distance
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

def match_features(des1, des2):
    # Find matches between descriptors of two images
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

# Use RANSAC to filter matches and find the best homography
def ransac_filter(matches, kp1, kp2):
    # Get matched points
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Compute homography with RANSAC
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # Filter out inliers
    inliers = [m for i, m in enumerate(matches) if mask[i]]
    return M, inliers

# Stitch images by warping I1 to I2
def stitch_images(img1, img2, M):
    # Warp img1 to align with img2
    warped_img1 = cv2.warpPerspective(img1, M, (img1.shape[1] + img2.shape[1], img1.shape[0]))
    
    # Place img2 in the warped image area
    warped_img1[0:img2.shape[0], 0:img2.shape[1]] = img2
    return warped_img1

# Sequentially stitch all images
panorama = images[0]
for i in range(1, len(images)):
    # Get descriptors and keypoints of consecutive image pairs
    kp1, des1 = all_keypoints[i - 1], all_descriptors[i - 1]
    kp2, des2 = all_keypoints[i], all_descriptors[i]
    
    # Step 2: Match features
    matches = match_features(des1, des2)
    
    # Visualize initial matches
    match_img = cv2.drawMatches(images[i - 1], kp1, images[i], kp2, matches[:50], None, flags=2)
    cv2.imwrite(f"{output_folder}/set4_matches.jpg", match_img)
    
    # Step 3: Apply RANSAC to refine matches
    M, inliers = ransac_filter(matches, kp1, kp2)
    
    # Visualize best matches after RANSAC
    inlier_img = cv2.drawMatches(images[i - 1], kp1, images[i], kp2, inliers, None, flags=2)
    cv2.imwrite(f"{output_folder}/set4_bestMatches.jpg", inlier_img)
    
    # Step 4: Warp and blend images
    panorama = stitch_images(panorama, images[i], M)

# Save final panorama
cv2.imwrite(f"{output_folder}/set4_panorama.jpg", panorama)


True

In [3]:
import cv2
import numpy as np
import os
import random

# Define paths for the image and result output
image_path = 'C:/Users/Haseeb Raza/Desktop/MIC_3/set4/set4_11_100_AML.png'
output_folder = 'C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4'
os.makedirs(output_folder, exist_ok=True)

# Load the image
image = cv2.imread(image_path)

# Initialize SIFT detector with max 800 keypoints
noOfmaxKeypts = 800
sift = cv2.xfeatures2d.SIFT_create(noOfmaxKeypts)

# Detect keypoints and compute descriptors
keypoints, descriptors = sift.detectAndCompute(image, None)

# Select 200 random keypoints for visualization
random_keypoints = random.sample(keypoints, min(200, len(keypoints)))

# Draw the 200 random keypoints on the image
image_with_keypoints = cv2.drawKeypoints(
    image, random_keypoints, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS
)

# Save the resulting image with the keypoints drawn
output_path = os.path.join(output_folder, 'set4_featurePoints.jpg')
cv2.imwrite(output_path, image_with_keypoints)

print(f"Image with 200 random feature points saved at {output_path}")


Image with 200 random feature points saved at C:/Users/Haseeb Raza/Desktop/MIC_3/Results/set4\set4_featurePoints.jpg
